# Apply Ranking Recovery Algorithms

In [1]:
import pandas as pd
import seaborn as sns

from fairpair import *

In [2]:
# create a FairPairGraph with disadvantaged minority
G = FairPairGraph()
G.generate_groups(15, 5)
G.group_assign_scores(nodes=G.majority_nodes, distr=Distributions.normal_distr)
G.group_assign_scores(nodes=G.minority_nodes, distr=Distributions.normal_distr, loc=0.3, scale=0.2) # give a disadvantage to the minority

In [3]:
# run 100 iterations of ProbKnockout sampling
sampler = ProbKnockoutSampling(G, warn=False)
sampler.apply(iter=100, k=1)
# apply davidScore for ranking recovery
ranker = RankRecovery(G)
ranking, other_nodes = ranker.apply(rank_using=davidScore) # by default, apply rankCentrality method
ranker._print_with_score(ranking) # sorted by rank score

,node,orig score,rank score
0,14,0.596430,9.868152
1,2,0.423971,9.577778
2,1,0.482185,9.363636
3,12,0.574583,7.291667
4,5,0.413708,6.539216
5,8,0.314513,-2.924896
6,10,0.333572,-2.924896
7,11,0.336343,-2.924896
8,9,0.182989,-6.310101
9,13,0.252433,-8.800654


## Metrics

In [4]:
# score-based metrics
print("Majority NDCG (by score):", ranker.calc_NDCG(G.majority, ranking)) # NDCG is only calculated as a "in-group" comparison because it is normalized
print("Minority NDCG (by score):", ranker.calc_NDCG(G.minority, ranking))
print("Majority MSE (by score):", ranker.calc_MSE(G.majority, ranking))
print("Minority MSE (by score):", ranker.calc_MSE(G.minority, ranking))

Majority NDCG (by score): 0.8574133040787919
Minority NDCG (by score): 0.9999298265692861
Majority MSE (by score): 0.2163303443296062
Minority MSE (by score): 0.05900863944774899


In [5]:
# rank-based matrics
print("Majority mean error (by rank):", ranker.calc_rank_mean_error(G.majority, ranking))
print("Minority mean error (by rank):", ranker.calc_rank_mean_error(G.minority, ranking))
print("Majority MSE (by rank):", ranker.calc_rank_MSE(G.majority, ranking))
print("Minority MSE (by rank):", ranker.calc_rank_MSE(G.minority, ranking))

Majority mean error (by rank): 1.0
Minority mean error (by rank): 2.4
Majority MSE (by rank): 34.714285714285715
Minority MSE (by rank): 7.6
